In [1]:
import os
import pandas as pd
import load_geolife
import add_geohash
import create_nodes
import geohash_location_count
import geohash_user_count
import create_geohash_meet
import create_geohash_meet_time

In [2]:
def create_dirs():
    if not os.path.isdir('images/'):
        os.makedirs('images/')
    if not os.path.isdir('htmls/'):
        os.makedirs('htmls/')
    if not os.path.isdir('data/'):
        os.makedirs('data/')

In [3]:
def removeDuplicates():
    duplicates = pd.read_csv('data/geolife_duplicates_content.csv')
    df = pd.read_csv('data/geolife_geohash_size_8.csv')
    df['px'] = df['Person ID'].astype('string') + "_" + df['Trajectory'].astype('string') + ".plt"
    duplicates['px'] = duplicates['user'].astype('string')+"_"+duplicates['file_name'].astype('string')
    noduplicates = df[~df['px'].isin(duplicates['px'])].copy(deep=True)
    noduplicates.to_csv('data/geolife_geohash_size_8_no_duplicates.csv',index=False)


In [4]:
create_dirs()
load_geolife.load_geolife('./geolife/Data/')
add_geohash.add_geohash()

removeDuplicates()
create_nodes.create_nodes()
geohash_location_count.geohash_location_count()
geohash_user_count.geohash_user_count()
count_files = create_geohash_meet.createGeohashMeetJsons()
create_geohash_meet.createGeohashMeetCSV(count_files)

# create_geohash_meet_time.create_geohash_time_meets()

  0%|          | 0/182 [00:00<?, ?it/s]

100%|██████████| 18165801/18165801 [08:47<00:00, 34445.00it/s]


loaded
415946
grouping...
grouped
1.0 415946 / 415946


100%|██████████| 414417/414417 [55:03<00:00, 125.46it/s] 


# Local

In [ ]:
import pandas as pd
from tqdm import tqdm
import json

def create_geohash_time_meets():
    df = pd.read_csv('data/geolife_geohash_meet_intervals_size_8.csv')
    df['meets_intervals'] = ""

    for index in tqdm(df.index):
        meets = json.loads(df.loc[index, 'meets_intervals'])
        cols = json.loads(df.loc[index, 'meets_intervals_keys'])
        all_intervals = {}
        for i in meets.keys():
            times = meets[i]
            interval = []
            first_time = [vi for vi in times[0]]
            prev_time = [vi for vi in times[0]]
            for time in times:
                if time[cols.index('Index')] - prev_time[cols.index('Index')] > 1:
                    interval.append([
                        prev_time[cols.index('Person ID')],
                        first_time[cols.index('Timestamp')],
                        prev_time[cols.index('Timestamp')],
                        prev_time[cols.index('Geohash')],
                        first_time[cols.index('Index')],
                        prev_time[cols.index('Index')],
                        ])
                    first_time = [vi for vi in time]
                prev_time = [vi for vi in time]

            interval.append([
                prev_time[cols.index('Person ID')],
                first_time[cols.index('Timestamp')],
                prev_time[cols.index('Timestamp')],
                prev_time[cols.index('Geohash')],
                first_time[cols.index('Index')],
                prev_time[cols.index('Index')],
            ])
            all_intervals[i]=interval
        df.loc[index, 'meets_intervals'] = json.dumps(all_intervals)
        df.loc[index, 'meets_intervals_keys'] = json.dumps(["Person ID", "Start Time", "End Time", "Geohash", "Start Index", "End Index"])
    df.to_csv('data/geolife_geohash_meet_intervals_size_8.csv',index=False)

# Global